https://pytorch.org/vision/stable/feature_extraction.html

In [ ]:
import torch
from torchvision.models import resnet50
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor
from torchvision.models.detection.mask_rcnn import MaskRCNN
from torchvision.models.detection.backbone_utils import LastLevelMaxPool
from torchvision.ops.feature_pyramid_network import FeaturePyramidNetwork

In [ ]:
# Чтобы помочь вам в разработке средства извлечения компонентов, вы можете распечатать его
# список доступных узлов для resnet 50.
m = resnet50()
print("model", m)

model ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), strid

In [ ]:
train_nodes, eval_nodes = get_graph_node_names(resnet50())
for val in train_nodes:
    print("train_nodes : ", val)
print("eval nodes : ", eval_nodes)

train_nodes :  x
train_nodes :  conv1
train_nodes :  bn1
train_nodes :  relu
train_nodes :  maxpool
train_nodes :  layer1.0.conv1
train_nodes :  layer1.0.bn1
train_nodes :  layer1.0.relu
train_nodes :  layer1.0.conv2
train_nodes :  layer1.0.bn2
train_nodes :  layer1.0.relu_1
train_nodes :  layer1.0.conv3
train_nodes :  layer1.0.bn3
train_nodes :  layer1.0.downsample.0
train_nodes :  layer1.0.downsample.1
train_nodes :  layer1.0.add
train_nodes :  layer1.0.relu_2
train_nodes :  layer1.1.conv1
train_nodes :  layer1.1.bn1
train_nodes :  layer1.1.relu
train_nodes :  layer1.1.conv2
train_nodes :  layer1.1.bn2
train_nodes :  layer1.1.relu_1
train_nodes :  layer1.1.conv3
train_nodes :  layer1.1.bn3
train_nodes :  layer1.1.add
train_nodes :  layer1.1.relu_2
train_nodes :  layer1.2.conv1
train_nodes :  layer1.2.bn1
train_nodes :  layer1.2.relu
train_nodes :  layer1.2.conv2
train_nodes :  layer1.2.bn2
train_nodes :  layer1.2.relu_1
train_nodes :  layer1.2.conv3
train_nodes :  layer1.2.bn3
train_

In [ ]:
# Возвращенные списки - это имена всех узлов графа (в порядке выполнения
#) для входной модели, отслеживаемой в режиме train и в режиме eval
# соответственно. Вы обнаружите, что `train_nodes` и `eval_nodes` совпадают
# для этого примера. Но если модель содержит поток управления, зависящий
# от режима обучения, они могут отличаться.

# Чтобы указать узлы, которые вы хотите извлечь, вы можете выбрать конечный узел
#, который отображается в каждом из основных слоев:
return_nodes = {
    # имя_узла: указанный пользователем ключ для вывода dict
    'layer1.2.relu_2': 'layer1',
    'layer2.3.relu_2': 'layer2',
    'layer3.5.relu_2': 'layer3',
    'layer4.2.relu_2': 'layer4',
}

# return_nodes = {
#     # имя_узла: указанный пользователем ключ для вывода dict
#     'layer1.2.relu_2': 'layer1',
# }

# Но `create_feature_extractor` также может принимать спецификации усеченных узлов
# # например, "уровень 1", поскольку он просто выберет последний узел, который является потомком
# спецификации. (Совет: будьте осторожны с этим, особенно если слой
# имеет несколько выходных данных. Не всегда гарантируется, что последняя выполненная операция
# будет соответствовать желаемому результату. Вы должны
# для подтверждения обратитесь к исходному коду входной модели.)
# return_nodes = {
#     'layer1': 'layer1',
#     'layer2': 'layer2',
#     'layer3': 'layer3',
#     'layer4': 'layer4',
# }


In [ ]:
#Создает новый модуль graph, который возвращает промежуточные узлы из заданной модели в виде словаря с указанными пользователем ключами в виде строк
#и запрошенными выходными данными в виде значений. Это достигается путем перезаписи графика вычислений модели с помощью FX,
#чтобы вернуть нужные узлы в качестве выходных данных.
#Все неиспользуемые узлы удаляются вместе с соответствующими им параметрами.

# Теперь вы можете создать средство извлечения объектов. Это возвращает модуль, чей метод forward
# возвращает словарь, подобный:
# {
#     'layer1': output of layer 1,
#     'layer2': output of layer 2,
#     'layer3': output of layer 3,
#     'layer4': output of layer 4,
# }
create_feature_extractor(m, return_nodes=return_nodes)

# print(m)
# print("\n\n\n\n\n\n")
# m2 = resnet50()
# print(m2)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Module(
    (0): Module(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (downsample): Module(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=Fal

In [ ]:
# Давайте соберем все это вместе, чтобы обернуть resnet 50 с помощью MaskRCNN
# Для MaskRCNN требуется магистраль с подключенным FPN
class Resnet50WithFPN(torch.nn.Module):
    def __init__(self):
        super(Resnet50WithFPN, self).__init__()

        # Get a resnet50 backbone
        m = resnet50()
        # Извлеките 4 основных слоя (примечание: для MaskRCNN требуется именно это имя
        # сопоставление для возвращаемых узлов)
        self.body = create_feature_extractor(
            m, return_nodes={f'layer{k}': str(v)
                             for v, k in enumerate([1, 2, 3, 4])})
        print("resnet50 backbone", self.body, "n\n\n")

        # Предварительный прогон для получения количества каналов для FPN
        inp = torch.randn(2, 3, 224, 224)

        with torch.no_grad():
            out = self.body(inp)

        in_channels_list = [o.shape[1] for o in out.values()]
        print("in_channels_list", in_channels_list)

        # Build FPN: extra_blocks - дополнительные блоки
        self.out_channels = 256
        self.fpn = FeaturePyramidNetwork(
            in_channels_list, out_channels=self.out_channels,
            extra_blocks=LastLevelMaxPool())

    def forward(self, x):
        x = self.body(x)
        x = self.fpn(x)
        return x

In [ ]:
# Now we can build our model!
model = MaskRCNN(Resnet50WithFPN(), num_classes=91).eval()
print("\n\n\n\n", model)

resnet50 backbone ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Module(
    (0): Module(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (downsample): Module(
        (0): Conv2d(64, 256, kernel_size=(1, 1), strid